In [1]:
# pip install pandas


In [2]:
# pip install sql

In [3]:
# pip install yfinance

In [4]:
import pandas as pd
from sqlalchemy import create_engine
import yfinance as yf

In [5]:
engine = create_engine("sqlite:///TEST_DB.db")

In [6]:
df = yf.download("NDX", start="2022-05-01", end = "2022-11-14")

[*********************100%***********************]  1 of 1 completed


In [7]:
# df.to_sql("NDX",engine)

In [8]:
max_date = pd.read_sql("SELECT MAX(Date) FROM NDX",engine).values[0][0]

In [9]:
max_date

'2022-11-14 00:00:00.000000'

In [12]:
df[df.index > max_date].to_sql("NDX",engine, if_exists = "append")

0

In [13]:
pd.read_sql("NDX",engine)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2022-05-02,12864.830078,13083.570312,12716.629883,13075.849609,13075.849609,4918240000
1,2022-05-03,13045.349609,13178.740234,12982.700195,13089.900391,13089.900391,4547280000
2,2022-05-04,13110.299805,13556.669922,12887.530273,13535.709961,13535.709961,5498050000
3,2022-05-05,13344.790039,13344.790039,12710.589844,12850.549805,12850.549805,5308920000
4,2022-05-06,12776.860352,12926.580078,12522.769531,12693.530273,12693.530273,5324480000
...,...,...,...,...,...,...,...
132,2022-11-08,11039.580078,11191.900391,10897.530273,11059.500000,11059.500000,5134500000
133,2022-11-09,10971.429688,11012.660156,10790.349609,10797.549805,10797.549805,4977460000
134,2022-11-10,11350.419922,11612.200195,11235.740234,11605.959961,11605.959961,6433000000
135,2022-11-11,11604.360352,11840.070312,11535.559570,11817.009766,11817.009766,5867750000


In [23]:
def sql_importer(symbol,start = "2022-05-01"):
    try:
        max_date = pd.read_sql(f"SELECT MAX(Date) FROM {symbol}",engine).values[0][0]
        print(max_date)
        new_data = yf.download(symbol, start = pd.to_datetime(max_date))
        new_rows = data[new_data.index > max_date]
        new_rows.to_sql(symbol, engine, if_exists = "append")
        print(str(len(new_rows)) + "new rows imported to DB")
    except:
        new_data = yf.download(symbol, start = start)
        new_data.to_sql(symbol, engine)
        print(f"New table created for {symbol} with {str(len(new_data))}  rows")

In [24]:
sql_importer("AAPL")

[*********************100%***********************]  1 of 1 completed
New table created for AAPL with 138  rows


In [25]:
stocks = ["DJI","IBM","TSLA","MSFT","AMZN"]

In [27]:
for stock in stocks:
    sql_importer(stock)

[*********************100%***********************]  1 of 1 completed

1 Failed download:
- DJI: No data found for this date range, symbol may be delisted
New table created for DJI with 0  rows
[*********************100%***********************]  1 of 1 completed
New table created for IBM with 138  rows
[*********************100%***********************]  1 of 1 completed
New table created for TSLA with 138  rows
[*********************100%***********************]  1 of 1 completed
New table created for MSFT with 138  rows
[*********************100%***********************]  1 of 1 completed
New table created for AMZN with 137  rows
